In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
Pkg.precompile()

# Why are we here?

- Who here has heard of Julia?
- Who has used Julia before?
- Who has contributed to Julia itself or a package?
- Why are **you** interested in the language?

# What is Julia?

## "A New Approach to Technical Computing"

- Less than 10 years old - project started in 2012
- Growing use in scientific computing, data science, and machine learning
- Users in industry, government, and academia, all over the world
- **One-sentence pitch: code as fast to write as Python, as fast to run as C++**
- Actually, Julia can often be faster than C++
- Much easier to parallelize code across multiple cores, multiple nodes, or the GPU than Python
- Code is readable, easy to share with students, collaborators, and the public
- First class support for linear algebra, sparse matrices, and numerical computing
- The community is friendly and full of expertise from many domains
- Easy to learn if you already know Python
- Easy to use existing Python/C code from Julia, so you don't have to rewrite everything
- Very good support for testing, documenting, and benchmarking code
- **Julia is one-based indexed, this isn't changing, love it or leave it.**

## Julia is used in a variety of fields

### Physics
- A group in CCQ, including Matt Fishman, Miles Stoudenmire, and me, has rewritten the C++ ITensors library [in Julia](https://github.com/itensor/itensors.jl) and seen equivalent performance while adding features that would have been much harder to implement in C++ (like [GPU support](https://github.com/itensor/itensorsgpu.jl))
- [Yao.jl](https://github.com/QuantumBFS/Yao.jl), a state of the art quantum circuit simulator

### Astronomy
- [Celeste.jl](https://github.com/jeff-regier/Celeste.jl), massively parallel stellar classification 
- [Eric Agol's](https://faculty.washington.edu/agol/) group in exoplanet astronomy uses Julia
- [NextGen CMB lensing, with GPU support](https://github.com/marius311/CMBLensing.jl)
- [SkyCoords.jl](https://juliaastro.github.io/SkyCoords.jl/latest/#Performance-1)
- [LombScargle.jl](https://github.com/JuliaAstro/LombScargle.jl#performance)
- [Photometry.jl](https://juliaastro.github.io/Photometry.jl/dev/)

### Biology
- [BioJulia packages](https://github.com/BioJulia/) for reading/writing datastructures for genomes and proteins
- [BioSequences.jl performance blogpost](https://biojulia.net/post/seq-lang/)


### Climate Science
- [ClimateMachine.jl](https://github.com/CliMA/ClimateMachine.jl), using machine learning to create a next generation cimate model, all in Julia
- [Oceananigans.jl](https://github.com/CliMA/Oceananigans.jl), with a cool video example [here](https://www.youtube.com/watch?v=kpUrxnKKMjI)


## Drawbacks of using Julia

- Since the language is new, many people are unfamiliar with it and unwilling to switch
- Less well developed community than Python has - no "JuliaLadies", for example, or regional JuliaCons
- Onboarding and tutorial materials still lacking - a great opportunity for someone to get involved
- Fewer mature packages than Python or C++
- Tooling is less mature -- debuggers, IDEs, etc (this is improving quickly)

## Just how fast is this language?

We can look at two kinds of benchmarks: "micro", for small operations that are repeated many times (like `sqrt`) and "macro", for a complete simulation/data analysis.

![micro-benchmarks](https://julialang.org/assets/benchmarks/benchmarks.svg)

## Some simple examples of Julia code

- A `for` loop
- Broadcasting
- Defining custom types
- Multiple dispatch
- Type instability!

In [ ]:
# There are lots of ways to iterate through a collection

A = rand(512)
B = similar(A)
@show eltype(A)
@show typeof(B)
for i in 1:length(A)
    B[i] = sin(A[i])
end

for (i, aa) in enumerate(A) # probably recognize this from Python
    B[i] = sin(aa)
end

for i in 2:2:length(A) # let's slice!
    B[i] = cos(A[i])
end

Ar = reshape(A, (256, 2))
Br = reshape(B, (2, 256))

# can still use linear indices
B[:] = A[:]

# we can fill B using a "generator" (like a list comprehension)
B = [sin(a) for a in A]

# or we can even use broadcasting
B .= sin.(A)

### More on broadcasting and memory

Because Julia is memory-managed, it's better if we can avoid allocations in our code, especially in tight inner loops. We can use the basic `@time` macro to see if some code we wrote is allocating.

In [ ]:
@time B = copy(A) # allocating

@time B = sin.(A)

@time B .= sin.(A)

Broadcasting is a nice way to minimize the number of allocations in code, and Julia can *automatically* broadcast functions over arrays and custom types. Let's look at some examples and introduce Julia's type system.

In [ ]:
# define a custom *abstract* type
abstract type AbstractPoint{T, N} end

mutable struct Point{N, T} <: AbstractPoint{N, T}
    coords::NTuple{N,T} # a Tuple of N elements of type T
end

# let's construct some Points

my_2d_point = Point{2, Int}(1, 2)

my_3d_point = Point{3, Float64}(1.0, 2.0, 3.0)

The above didn't work, because we need to define a constructor for `Point`:

In [ ]:
mutable struct Point{N, T} <: AbstractPoint{N, T}
    coords::NTuple{N,T} # a Tuple of N elements of type T
    function Point{N, T}(coords::Vararg{T, N}) where {N, T}
        new(coords)
    end
end

# let's construct some Points

my_2d_point = Point{2, Int}(1, 2)

my_3d_point = Point{3, Float64}(1.0, 2.0, 3.0)


In [ ]:
@show my_2d_point isa Point
@show my_2d_point isa Point{2}
@show my_2d_point isa Point{Int}
@show my_2d_point isa Point{3, Int}

What if we don't know the coordinate type in advance?

In [ ]:
try
    my_2d_point = Point(1, 2)
catch e
    @warn e
end

Point(coords::Vararg{T, N}) where {N, T} = Point{N, T}(coords...)

my_2d_point = Point(1, 2)

### Why is this useful?

We can write custom methods for our types!

In [ ]:
function angle(point_a, point_b)
    θ_a = atan(point_a.coords[2]/point_a.coords[1])
    θ_b = atan(point_b.coords[2]/point_b.coords[1])
    θ_b - θ_a
end

# let's write some tests for this!
using Test
y_axis = Point(0, 1)
x_axis = Point(1, 0)
@test angle(x_axis, y_axis) == π/2
@test angle(y_axis, x_axis) == -π/2

This *seems* to work ... but what if I have a 3d point? Can't I define an angle between two 3-d rays? (What do the astronomers think about this idea?). In fact, we know that:

`a * b = |a||b|cosθ`

In [ ]:
function angle(point_a, point_b)
    inner_prod = sum(point_a.coords .* point_b.coords)
    mag_a = hypot(point_a.coords...)
    mag_b = hypot(point_b.coords...)
    acos(inner_prod/(mag_a*mag_b))
end
y_axis = Point(0, 1, 0)
x_axis = Point(1, 0, 0)
z_axis = Point(0, 0, 1)
@test angle(x_axis, y_axis) == π/2
@test angle(z_axis, y_axis) == π/2
@test angle(y_axis, x_axis) == π/2

# and let's run some timings

This method shouldn't work if we pass points with different dimensionality - one way to deal with this would be:

In [ ]:
Base.ndims(p::Point{N, T}) where {N, T} = N
using LinearAlgebra
function angle_notype(point_a, point_b)
    ndims(point_a) == ndims(point_b) || throw(DimensionMismatch("A and B have different dimensionality"))
    inner_prod = dot(point_a.coords, point_b.coords)
    mag_a = hypot(point_a.coords...)
    mag_b = hypot(point_b.coords...)
    acos(inner_prod/(mag_a*mag_b))
end
y_axis = Point(0, 1, 0)
x_axis = Point(1, 0, 0)
z_axis = Point(0, 0, 1)
@test angle(x_axis, y_axis) == π/2
@test angle(z_axis, y_axis) == π/2
@test angle(y_axis, x_axis) == π/2

@test_throws DimensionMismatch angle(Point(1, 2), Point(1, 2, 3))

In [ ]:
using LinearAlgebra
function angle_typed(point_a::Point{N, T}, point_b::Point{N, T}) where {N, T}
    inner_prod = dot(point_a.coords, point_b.coords)
    mag_a = hypot(point_a.coords...)
    mag_b = hypot(point_b.coords...)
    acos(inner_prod/(mag_a*mag_b))
end
y_axis = Point(0, 1, 0)
x_axis = Point(1, 0, 0)
z_axis = Point(0, 0, 1)
@test angle_typed(x_axis, y_axis) == π/2
@test angle_typed(z_axis, y_axis) == π/2
@test angle_typed(y_axis, x_axis) == π/2
@test_throws MethodError angle_typed(Point(1, 2), Point(1, 2, 3))

as = [Point(a...) for a in collect(zip(rand(512), rand(512)))]
bs = [Point(b...) for b in collect(zip(rand(512), rand(512)))]

@time angle_notype.(as, bs)
@time angle_typed.(as, bs)

## Multiple dispatch: an example with our points

In [ ]:
@show as isa Vector

# define a new "scalar multiplication" method for Point

Base.:*(x::Number, a::Point) = Point((x .* a.coords)...)
Base.:*(a::Point, x::Number) = Point((x .* a.coords)...)
Base.:+(a::Point{N, T}, b::Point{N, T}) where {N, T} = Point((a.coords .+ b.coords)...)

cs = [Point(c...) for c in collect(zip(rand(512), rand(512)))]
cns = rand(512)

ds = cns .* cs

es = Point.(rand(12, 12))
fs = rand(12, 12)

es * fs

In [ ]:
# We need to define what the zero of this type is
Base.zero(::Type{Point{N, T}}) where {N, T} = Point(ntuple(i->zero(T), N)...)
Base.zero(p::Point{N, T}) where {N, T}      = Point(ntuple(i->zero(T), N)...)

es * fs

In [ ]:
Base.:/(a::Point, x::Number) = Point((inv(x) .* a.coords)...)
es ./ 2.0

## Be careful about type instabilities!

In [ ]:
using BenchmarkTools

A64 = rand(Float64, 100_000)
A32 = rand(Float32, 100_000)
A8  = rand(Int8, 100_000)

function my_sin(A::AbstractArray{T}) where {T}
    s = 0
    for (ai, a) in enumerate(A)
        s += 1
        s /= sin(a)
    end
    return s
end

function my_sin2(A::AbstractArray{T}) where {T}
    s = zero(T)
    for (ai, a) in enumerate(A)
        s += one(T)
        s /= sin(a)
    end
    return s
end

@btime my_sin($A64)
@btime my_sin2($A64)

@btime my_sin($A32)
@btime my_sin2($A32)

@btime my_sin($A8)
@btime my_sin2($A8)

# how can we detect what's causing the difference?

In [ ]:
@code_warntype my_sin(A32)

In [ ]:
@code_warntype my_sin2(A32)

## It's easy to use libraries written in other languages

This way, you can use Julia for parts of the simulation you want to rewrite now, while still retaining the battle-tested library you've put many hours into developing.

### From Python

Julia has a very nice package called `PyCall` that lets you import Python libraries, use Python types, and call Python functions. Let's look at an example:

In [ ]:
using PyCall, Conda
Conda.add("geopy", channel="conda-forge")
gp = pyimport("geopy")
gcs = pyimport("geopy")."geocoders"
locator = gcs.Nominatim(user_agent="SF", timeout=nothing)

In [ ]:
FI = locator.geocode("162 5th Avenue NYC")
@show FI.address, FI.longitude, FI.latitude
SF = locator.geocode("160 5th Avenue NYC")
@show SF.address, SF.longitude, SF.latitude

dist = pyimport("geopy.distance")
distance = dist.geodesic((FI.longitude, FI.latitude), (SF.longitude, SF.latitude))

### From C

- Calling C from Julia (or Julia from C!) is natively supported, as is using C `struct`s, and all those good things. Julia and C can pass arrays back and forth, and it's simple to call any function in a shared object library from Julia.

In [ ]:
path = ccall(:getenv, Cstring, (Cstring,), "SHELL")
unsafe_string(path)

## Showcases

In this section we'll see some cool examples of things Julia and its packages are able to do:
- Plotting and differential equation solving - look in the `Animated plots.ipynb` notebook
- Parallelism - single node (`Single node parallelism.ipynb`) and multi-node (`distributed_computing.jl`)
- Native Julia machine learning - (`Machine learning in Julia.ipynb`)

## Resources to learn more

- There is a Julia [book](https://www.amazon.com/Julia-High-Performance-Avik-Sengupta/dp/178829811X)!
- Julia [Slack](https://julialang.org/slack/) is very welcoming and we have dedicated channels for physics, astronomy, biology, and areas like HPC
- We also have a [discourse forum](https://discourse.julialang.org/) for people to ask questions and get help
- The Julia [youtube page](https://www.youtube.com/user/JuliaLanguage) has many talks from previous JuliaCons, which often include Jupyter notebook tutorials
- [Learn X in Y minutes](https://learnxinyminutes.com/docs/julia/) - a basic introduction to Julia syntax